In [1]:
from astropy.table import Table
import pandas as pd
from gaiaxpy import calibrate
from gaiaxpy import convert
import time
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import yaml
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy import units as u
from scipy.ndimage import gaussian_filter1d
import sys
from progress.bar import Bar
from tqdm.notebook import tqdm
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u
from astropy.wcs.utils import skycoord_to_pixel
from astropy.wcs import WCS
import warnings
warnings.filterwarnings("ignore")
import os
from multiprocessing import Pool
from multiprocessing import cpu_count

In [2]:
def median_a(x,lw=5,lower=10000,wave=[]):
    if len(wave) > 0:
        index=np.where(wave < lower)[0]
        index2=np.where(wave >= lower)[0]
        x1=np.copy(x)
        x=x[index]
    x_n=np.zeros(len(x))
    for i in range(0, len(x)):
        if i <= lw:
            x_d=x[0:lw]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d)
        if i >= len(x)-lw:
            x_d=x[len(x)-lw:len(x)]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d)
        if i > lw and i < len(x)-lw:
            x_d=x[i-lw:i+lw]
            #x_d=reject_outliers(x_d)
            x_n[i]=np.nanmean(x_d) 
    if len(wave) > 0:
        x1[index]=x_n
        x1[index2]=x_n[-1]
        x_n=x1
    return x_n

In [3]:
def get_apertures(file):
    ra=[]
    dec=[]
    rad=[]
    colr=[]
    namet=[]
    l1=[]
    l2=[]
    th=[]
    typ=[]
    f=open(file,'r')
    ct=1
    for line in f:
        if not 'Region' in line and not 'fk5' in line and not 'global' in line:
            if 'circle' in line:
                data=line.replace('\n','').replace('circle(','').replace('") # color=',' , ').replace(' width=',' , ').replace(' text={',' , ').replace('}',' ')
                data=data.split(',')
                data=list(filter(None,data))
                #print(data)
                ra.extend([data[0]])
                dec.extend([data[1]])
                rad.extend([float(data[2])])
                colr.extend([data[3].replace(' ','')])
                try:
                    namet.extend([data[5].replace(' ','')])
                except:
                    namet.extend([str(int(ct))])
                l1.extend([np.nan])
                l2.extend([np.nan])
                th.extend([np.nan])    
                typ.extend(['circle'])
            if 'box' in line:
                data=line.replace('\n','').replace('box(','').replace(') # color=',' , ').replace(' width=',' , ').replace(' text={',' , ').replace('}',' ')
                data=data.split(',')
                data=list(filter(None,data))
                ra.extend([data[0]])
                dec.extend([data[1]])
                l1.extend([float(data[2].replace('"',''))])
                l2.extend([float(data[3].replace('"',''))])
                th.extend([float(data[4])])
                colr.extend([data[5].replace(' ','')])
                try:
                    namet.extend([data[7].replace(' ','')])
                except:
                    namet.extend([str(int(ct))])
                rad.extend([np.nan])    
                typ.extend(['box'])
            ct=ct+1
    ra=np.array(ra)
    dec=np.array(dec)
    rad=np.array(rad)
    colr=np.array(colr)
    namet=np.array(namet)
    typ=np.array(typ)
    l1=np.array(l1)
    l2=np.array(l2)
    th=np.array(th)
    return ra,dec,rad,l1,l2,th,colr,namet,typ

In [4]:
def extract_spec(spec,hdr,ra='',dec='',rad=1.5,pix=0.35,avgra=False):
    sky1=SkyCoord(ra+' '+dec,frame=FK5, unit=(u.hourangle,u.deg))
    val1=sky1.ra.deg
    val2=sky1.dec.deg
    wcs = WCS(hdr)
    wcs=wcs.celestial
    ypos,xpos=skycoord_to_pixel(sky1,wcs)
    print(xpos,ypos,'POS Pixel')
    val1=sky1.to_string('hmsdms')
    print(val1,'RA1,DEC1')
        
    nz,nx,ny=spec.shape
    radis=np.zeros([nx,ny])
    for i in range(0, nx):
        for j in range(0, ny):
            x_n=i-xpos
            y_n=j-ypos
            r_n=np.sqrt((y_n)**2.0+(x_n)**2.0)*pix
            radis[i,j]=r_n
    single_T=np.zeros(nz)
    nt=np.where(radis <= rad)
    for i in range(0, nz):
        tmp=spec[i,:,:]
        tmp[np.where(tmp <= 0)]=np.nan
        if avgra:
            single_T[i]=np.nanmean(tmp[nt])
        else:
            single_T[i]=np.nansum(tmp[nt])
        
    crpix=hdr["CRPIX3"]
    try:
        cdelt=hdr["CD3_3"]
    except:
        cdelt=hdr["CDELT3"]
    crval=hdr["CRVAL3"]
    wave_f=(crval+cdelt*(np.arange(nz)+1-crpix))*1e10
    
    return wave_f,single_T,xpos,ypos

In [5]:
def sycall(comand):
    import os
    linp=comand
    os.system(comand)
    
def task_wrapper(args):
    return ifu_const(*args)
    
def ifu_const(spec_ifu,specE_ifu,x_ifu_V,y_ifu_V,fibA,pix_s,yo,xi,xf,nw,nl,npros,nproc):
    val=int(nl/nproc)
    a1=val*npros
    if npros < nproc-1:
        a2=val*(npros+1)
    else:
        a2=nl
    spec_fint=[]
    specE_fint=[]
    if len(x_ifu_V.shape) > 1:
        for j in range(a1, a2):
            yi=yo+pix_s*j
            yf=yo+pix_s*(j+1)
            spt_new=np.zeros(nw)
            sptE_new=np.zeros(nw)
            Wgt=np.zeros(nw)
            Rspt=np.sqrt((y_ifu_V[:,0]-(yf+yi)/2.0)**2.0)
            ntpt=np.where(Rspt <= (fibA*3.5*2/2.0))[0]
            x_ifu_Vt=x_ifu_V[ntpt,:]
            y_ifu_Vt=y_ifu_V[ntpt,:]
            spec_ifut=spec_ifu[ntpt,:]
            specE_ifut=specE_ifu[ntpt,:]
            for k in range(0, len(x_ifu_Vt[:,0])):
                Rsp=np.sqrt((x_ifu_Vt[k,:]-(xf+xi)/2.0)**2.0+(y_ifu_Vt[k,:]-(yf+yi)/2.0)**2.0)
                ntp=np.where(Rsp <= (fibA*3.5/2.0))
                Wg=np.zeros(nw)
                if len(ntp[0]) > 0:   
                    Wg[ntp]=np.exp(-(Rsp[ntp]/pix_s)**2.0/2.0)
                    spt_new[ntp]=spec_ifut[k,ntp]*Wg[ntp]+spt_new[ntp]
                    sptE_new[ntp]=specE_ifut[k,ntp]*Wg[ntp]+sptE_new[ntp]
                Wgt=Wgt+Wg
            ntp=np.where(Wgt == 0)
            if len(ntp[0]) > 0:
                Wgt[ntp]=1
            spec_fint.extend([spt_new/Wgt])
            specE_fint.extend([sptE_new/Wgt])
    else:
        for j in range(a1, a2):
            yi=yo+pix_s*j
            yf=yo+pix_s*(j+1)
            spax_new=0
            spaxE_new=0
            Wgt=0
            for k in range(0, len(x_ifu_V)):
                Rsp=np.sqrt((x_ifu_V[k]-(xf+xi)/2.0)**2.0+(y_ifu_V[k]-(yf+yi)/2.0)**2.0)
                if Rsp <= (fibA*3.5/2.0):   
                    Wg=np.exp(-(Rsp/pix_s)**2.0/2.0)
                    spax_new=spec_ifu[k]*Wg+spax_new
                    spaxE_new=specE_ifu[k]*Wg+spaxE_new
                    Wgt=Wgt+Wg
            if Wgt == 0:
                Wgt=1
            spec_fint.extend([spax_new/Wgt])
            specE_fint.extend([spaxE_new/Wgt])
    return ([spec_fint,specE_fint])
    
def rotate(xx,yy,angle):
    # rotate x and y cartesian coordinates by angle (in degrees)
    # about the point (0,0)
    theta = -1.*angle * np.pi / 180. # in radians
    xx1 = np.cos(theta) * xx - np.sin(theta) * yy
    yy1 = np.sin(theta) * xx + np.cos(theta) * yy
    return xx1, yy1

def make_radec(xx0,yy0,ra,dec,pa):
    xx, yy = rotate(xx0,yy0,pa)
    ra_fib = ra*3600.0 + xx/np.cos(dec*np.pi/180.) 
    dec_fib = dec*3600.0 + yy#- 
    return ra_fib, dec_fib    
    
def map_ifu(expnumL,multiT=False,agcam_dir='',redux_dir='',mjd=['0000'],redux_ver='0.1.1.dev0/1111/',scp=112.36748321030637,basename='lvmCFrame-NAME.fits',path_lvmcore=''):
    file=path_lvmcore+'/metrology/lvm_fiducial_fibermap.yaml'
    f=open(file,'r')
    fiber_map = yaml.safe_load(f)
    fiber_map=fiber_map['fibers']
    xp=[]
    yp=[]
    Std_id=[]
    for i in range(0, len(fiber_map)):
        if 'science' in fiber_map[i][4]:
            xp.extend([float(fiber_map[i][8])*scp])
            yp.extend([float(fiber_map[i][9])*scp])
            Std_id.extend([int(fiber_map[i][0])-1])
    xp=np.array(xp)
    yp=np.array(yp)
    try:
        nlt=len(expnumL)
    except:
        nlt=1
    for i in range(0, nlt):
        if nlt == 1:
            expnum=expnumL[i]
        else:
            expnum=expnumL[i]
        expn=str(int(expnum)).zfill(8)
        file=redux_dir+'/'+redux_ver+'/'+mjd[i % len(mjd)]+'/'+basename.replace('NAME',expn)
        try:
            [rss, hdr0]=fits.getdata(file,'FLUX', header=True)
            hdr1=fits.getheader(file,0)
            [erss, hdre]=fits.getdata(file,'ERROR', header=True)
        except:
            [rss, hdr0]=fits.getdata(file,0, header=True)
            [erss, hdre]=fits.getdata(file,1, header=True)
        crpix=hdr0["CRPIX1"]
        cdelt=hdr0["CDELT1"]
        crval=hdr0["CRVAL1"]
        expT=float(hdr1['EXPTIME'])
        #rss=rss/expT
        #erss=erss/expT
        
        agcam_coadd = agcam_dir+'/'+mjd[i % len(mjd)]+'/coadds/'+'lvm.sci.coadd_s'+expnum+'.fits'
        if os.path.isfile(agcam_coadd):
            agcam_hdu = fits.open(agcam_coadd)
            agcam_hdr = agcam_hdu[1].header
            w = WCS(agcam_hdr)
            cen = w.pixel_to_world(2500,1000)
            rac = cen.ra.deg  #agcam_hdr['RAMEAS']
            dec = cen.dec.deg #agcam_hdr['DECMEAS']hdr0
            PA = agcam_hdr['PAMEAS'] - 180.
            agcam_hdu.close()
        else:
            rac=hdr1["POSCIRA"]
            dec=hdr1["POSCIDE"]
            PA=hdr1["POSCIPA"]
    
        if i == 0:
            rac0=rac
            dec0=dec
            nx0,ny0=rss.shape
            wave0=crval+cdelt*(np.arange(ny0)+1-crpix)
            nfib0=len(Std_id)  
            rss_f=np.zeros([nfib0*nlt,ny0])
            rss_ef=np.zeros([nfib0*nlt,ny0])
            rss_f[0:nfib0,:]=rss[Std_id,:]
            rss_ef[0:nfib0,:]=erss[Std_id,:]
            x_ifu_V=np.zeros([nfib0*nlt,ny0])
            y_ifu_V=np.zeros([nfib0*nlt,ny0])
            for k in range(0, ny0):
                ra_fib, dec_fib=make_radec(xp,yp,rac,dec,PA)
                x_ifu_V[0:nfib0,k]=ra_fib#xp+rac*3600
                y_ifu_V[0:nfib0,k]=dec_fib#yp+dec*3600
        else:
            nx,ny=rss.shape  
            wave=crval+cdelt*(np.arange(ny)+1-crpix)
            for j in range(0, nfib0):
                rss_f[nfib0*i+j,:]=interp1d(wave,rss[Std_id[j],:],kind='linear',bounds_error=False)(wave0)
                rss_ef[nfib0*i+j,:]=interp1d(wave,erss[Std_id[j],:],kind='linear',bounds_error=False)(wave0)
            for k in range(0, ny0):
                ra_fib, dec_fib=make_radec(xp,yp,rac,dec,PA)
                x_ifu_V[nfib0*i:nfib0*(i+1),k]=ra_fib#xp+rac*3600
                y_ifu_V[nfib0*i:nfib0*(i+1),k]=dec_fib#yp+dec*3600  
        
    yot=(np.amax(y_ifu_V[:,0])+np.amin(y_ifu_V[:,0]))/2.0
    xot=(np.amax(x_ifu_V[:,0])+np.amin(x_ifu_V[:,0]))/2.0
    x_ifu_V=x_ifu_V-xot
    y_ifu_V=y_ifu_V-yot
    nw=len(wave0)
    ns=len(x_ifu_V[:,0])
    pix_s=18.5#/2.
    fibA=35.3
    thet=0.0
    nl=int(round((np.amax([np.amax(x_ifu_V[:,0]),-np.amin(x_ifu_V[:,0]),np.amax(y_ifu_V[:,0]),-np.amin(y_ifu_V[:,0])])+1)*2/pix_s))
    #nl=int(nl*0.6)
    ifu=np.zeros([nw,nl,nl])
    ifu_e=np.ones([nw,nl,nl])
    ifu_1=np.ones([nw,nl,nl])
    ifu_m=np.zeros([nw,nl,nl])
    xo=-nl/2*pix_s
    yo=-nl/2*pix_s
    xi=xo
    xf=xo
    facto=(pix_s)**2.0/(np.pi*(fibA/2.0)**2.0)
    spec_ifu=rss_f*facto
    specE_ifu=rss_ef*facto 
    #bar = Bar('Processing', max=nl)
    #from multiprocessing import Pool
    from multiprocessing.pool import ThreadPool
    pbar=tqdm(total=nl)
    int_spect=np.zeros(nw)
    for i in range(0, nl):
        xi=xf
        xf=xf+pix_s
        Rsp=np.sqrt((x_ifu_V[:,0]-(xf+xi)/2.0)**2.0)
        ntp=np.where(Rsp <= (fibA*3.5*2/2.0))[0]
        if multiT:
            nproc=3#3#cpu_count()
            with ThreadPool(nproc) as pool:
                args=[(spec_ifu[ntp,:],specE_ifu[ntp,:],x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,yo,xi,xf,nw,nl,npros,nproc) for npros in range(0, nproc)]                    
                #result_l = pool.starmap(ifu_const, args)
                result_l = pool.map(task_wrapper, args)
                #async_results = [pool.apply_async(ifu_const, args=(spec_ifu,x_ifu_V,y_ifu_V,fibA,pix_s,yo,xi,xf,nw,nl,npros,nproc)) for npros in range(0, nproc)]
                #result_l = [ar.get() for ar in async_results]
        else:
            nproc=1
            npros=0
            result_l=[]
            args=(spec_ifu[ntp,:],specE_ifu[ntp,:],x_ifu_V[ntp,:],y_ifu_V[ntp,:],fibA,pix_s,yo,xi,xf,nw,nl,npros,nproc)
            result_l.extend([task_wrapper(args)])
        for npros in range(0, nproc):
            result=result_l[npros]
            val=int(nl/nproc)
            a1=val*npros
            if npros < nproc-1:
                a2=val*(npros+1)
            else:
                a2=nl
            ct=0
            for j in range(a1, a2):
                ifu[:,j,i]=result[0][ct]
                ifu_e[:,j,i]=result[1][ct]
                ct=ct+1
        
        #bar.next()
        pbar.update(1)
    #bar.finish()
    pbar.close()
    
    h1=fits.PrimaryHDU(ifu)
    h2=fits.ImageHDU(ifu_e)
    h3=fits.ImageHDU(ifu_1)
    h4=fits.ImageHDU(ifu_m)
    head_list=[h1,h2,h3,h4]

    dx=0
    dy=0
    h=h1.header
    keys=list(hdr0.keys())
    for i in range(0, len(keys)):
        if not "COMMENT" in  keys[i] and not 'HISTORY' in keys[i]:
            h[keys[i]]=hdr0[keys[i]]
            h.comments[keys[i]]=hdr0.comments[keys[i]]
    #del h["CDELT1"]
    del h["WCSAXES"]
    h["NAXIS"]=3
    h["NAXIS3"]=nw 
    h["NAXIS1"]=nl
    h["NAXIS2"]=nl
    #h["NDITER"]=(len(files),'Number of dither observations')
    #h["BUNIT"]= ('1E-16 erg/s/cm^2','Unit of pixel value ' )
    #h["OBJECT"]=hdr_0[0]['OBJECT']
    #h["CTYPE"] = ("RA---TAN", "DEC--TAN")
    h["CRVAL1"]=xot/3600.0#hdr1['CRVAL1']
    h["CD1_1"]=-np.cos(thet*np.pi/180.)*pix_s/3600.
    h["CD1_2"]=-np.sin(thet*np.pi/180.)*pix_s/3600.
    h["CRPIX1"]=nl/2+dx
    h["CTYPE1"]='RA---TAN'
    h["CRVAL2"]=yot/3600.0#hdr1['CRVAL2']
    h["CD2_1"]=-np.sin(thet*np.pi/180.)*pix_s/3600.
    h["CD2_2"]=np.cos(thet*np.pi/180.)*pix_s/3600.
    h["CRPIX2"]=nl/2+0.5+dy
    h["CTYPE2"]='DEC--TAN'
    h["CUNIT1"]='deg     '                                           
    h["CUNIT2"]='deg     '
    h["CDELT3"]=cdelt*1e10 
    #h["CD3_3"]=cdelt*1e10 
    h["CRPIX3"]=crpix
    h["CRVAL3"]=crval*1e10
    h["CUNIT3"]=('Angstrom','Units of coordinate increment and value    ')    
    h["CTYPE3"]=('WAVE    ','Air wavelength (linear) ')
    h["RADESYS"]='FK5     '
    h["OBJSYS"]='ICRS    '
    h["EQUINOX"]=2000.00
    h["IFUCON"]=(str(int(ns))+' ','NFibers')
    h["BUNIT"]='erg/s/cm^2'
    hlist=fits.HDUList(head_list)
    hlist.update_extend()
    basenameC='lvmCube-NAME.fits'
    file=redux_dir+'/'+redux_ver+'/'+mjd[len(mjd)-1]+'/'+basenameC.replace('NAME',expn)
    out_fit=file
    hlist.writeto(out_fit,overwrite=True)
    sycall('gzip -f '+out_fit)

In [6]:
path_lvmcore=os.environ['LVMCORE_DIR']
path_sas=os.environ['SAS_BASE_DIR']
explist=['00005511','00005512']#,
explist=['00005513','00005514']
mjd=['60212']
#explist=['00006109','00006110','00006111','00006112','00006113','00006114','00006115','00006445','00006446','00006447','00006448','00006449','00006450','00006634','00006635','00006636','00006637','00006638','00006639']
#mjd=['60222','60222','60222','60222','60222','60222','60222','60228','60228','60228','60228','60228','60228','60231','60231','60231','60231','60231','60231']
explist=['00005298','00006167','00006168','00006169','00006241','00006243','00006244','00006245','00006365','00006366','00006372','00006817']#,'00006818','00006819']
mjd=['60207','60223','60223','60223','60224','60224','60224','60224','60226','60226','60226','60235']#,'60235','60235']
explist=['00004285','00004289','00004291','00005020','00006421','00006422','00006425','00006426','00006587','00006588','00006659','00006660','00006840','00006841','00006843']
mjd=['60191','60191','60191','60203','60228','60228','60228','60228','60231','60231','60232','60232','60236','60236','60236']
#explist=['00006100']#,'00006101','00006102','00006103','00006104','00006105','00006106','00006107','00006108']
#mjd=['60222']
map_ifu(explist,multiT=True,path_lvmcore=path_lvmcore,agcam_dir=path_sas+'/sdsswork/data/agcam/lco',redux_dir=path_sas+'/sdsswork/lvm/spectro/redux',mjd=mjd,redux_ver='0.1.1.dev0/1111',basename='lvmCFrame-NAME.fits')

  0%|          | 0/263 [00:00<?, ?it/s]